<a href="https://colab.research.google.com/github/Harry926/twitter_relation/blob/main/2022_5_4_%E5%96%9C%E6%80%92%E5%93%80%E5%90%84200%E7%AD%86bert%E8%A8%93%E7%B7%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')
data = pd.read_csv("/content/gdrive/My Drive/200_喜怒哀.csv") #記得把資料丟到雲端硬碟

Mounted at /content/gdrive


In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 12.0 MB/s 
     |████████████████████████████████| 6.6 MB 40.0 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 49.6 MB/s 
     |████████████████████████████████| 880 kB 47.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b8c22e082fc462b220fbe390de6ad24fe86fbf9de266e00dfc3e183d8bdd83f7
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import AdamW
from transformers import BertForSequenceClassification

class Dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]

token = BertTokenizer.from_pretrained('bert-base-chinese')

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [4]:
train_text = data['text'].to_numpy()
train_target = data['target'].to_numpy()

train_dataset = Dataset(train_text, train_target)

#test_text = 晚點生.to_numpy()
#test_target = 晚點生.to_numpy()

#test_dataset = Dataset(test_text, test_target)

In [5]:
def collate_fn(data):
    sents = [data[i][0] for i in range(len(data))]
    labels = [data[i][1] for i in range(len(data))]
    
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                    truncation=True, 
                    padding='max_length',
                    max_length=50,
                    return_tensors='pt',
                    return_length=True)
    
    input_ids = data['input_ids'] 
    attention_mask = data['attention_mask'] 
    token_type_ids = data['token_type_ids'] 
    labels = torch.LongTensor(labels) 
    
    return input_ids, attention_mask, token_type_ids, labels

In [6]:
trainloader = DataLoader(dataset=train_dataset,
              batch_size=1,
              collate_fn=collate_fn,
              shuffle=True,
              drop_last=True)
"""testloader = DataLoader(dataset=test_dataset,
              batch_size=1,
              collate_fn=collate_fn,
              shuffle=True,
              drop_last=True)"""

'testloader = DataLoader(dataset=test_dataset,\n              batch_size=1,\n              collate_fn=collate_fn,\n              shuffle=True,\n              drop_last=True)'

In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=3)

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        for data in dataloader:
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
          
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)

            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()

            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))  
    
    if compute_acc:
        if total != 0:
            acc = correct / total
        else:
            acc = correct
        return predictions, acc
    return predictions
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda:0


In [9]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量：102269955
線性分類器的參數量：2307



In [10]:
model = model.to(device)

model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

EPOCHS = 6
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]

        optimizer.zero_grad()
        
        outputs = model(input_ids=tokens_tensors, 
                  token_type_ids=segments_tensors, 
                  attention_mask=masks_tensors, 
                  labels=labels)

        loss = outputs[0]

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))
    path_name = "2022.5.4_" + str(epoch + 1) + "_model.pth"
    torch.save(model, path_name)

[epoch 1] loss: 579.228, acc: 0.840
[epoch 2] loss: 235.084, acc: 0.967
[epoch 3] loss: 99.714, acc: 0.985
[epoch 4] loss: 27.534, acc: 0.998
[epoch 5] loss: 16.103, acc: 0.998
[epoch 6] loss: 14.228, acc: 1.000
